In [14]:
import pandas as pd
%pip install requests
import requests
# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head(100)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-11T16:10:27.350Z,61.549100,-148.342200,9.300,1.50,ml,NaN,NaN,NaN,0.78,...,2025-11-11T16:12:25.472Z,"28 km SSE of Chickaloon, Alaska",earthquake,NaN,0.300,NaN,NaN,automatic,ak,ak
1,2025-11-11T15:54:30.340Z,34.195000,-117.481333,3.390,2.36,ml,133.0,26.0,0.019800,0.23,...,2025-11-11T16:26:19.405Z,"7 km SSE of Lytle Creek, CA",earthquake,0.12,0.300,0.181000,25.0,automatic,ci,ci
2,2025-11-11T15:50:43.840Z,38.790833,-122.760498,1.350,1.12,md,11.0,74.0,0.009797,0.02,...,2025-11-11T16:07:18.020Z,"2 km NNW of The Geysers, CA",earthquake,0.28,0.550,0.160000,10.0,automatic,nc,nc
3,2025-11-11T15:50:19.791Z,60.688500,-150.564400,35.500,2.30,ml,NaN,NaN,NaN,0.74,...,2025-11-11T15:57:33.048Z,"20 km NNE of Sterling, Alaska",earthquake,NaN,1.300,NaN,NaN,automatic,ak,ak
4,2025-11-11T15:42:48.304Z,64.973800,-147.366200,7.300,1.40,ml,NaN,NaN,NaN,0.32,...,2025-11-11T15:45:12.793Z,"12 km E of Fox, Alaska",earthquake,NaN,0.800,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2025-11-11T01:55:41.690Z,38.687668,-122.500168,6.560,1.44,md,9.0,101.0,0.068790,0.06,...,2025-11-11T03:37:18.945Z,"12 km SE of Middletown, CA",earthquake,0.38,1.440,0.180000,9.0,automatic,nc,nc
96,2025-11-11T01:54:25.075Z,39.215400,28.014400,15.115,4.10,mb,25.0,45.0,0.874000,0.85,...,2025-11-11T03:57:35.040Z,"14 km WSW of Sındırgı, Turkey",earthquake,3.86,5.719,0.214000,6.0,reviewed,us,us
97,2025-11-11T01:50:23.990Z,44.677333,-110.664500,7.330,1.08,ml,19.0,85.0,0.036840,0.15,...,2025-11-11T14:50:30.840Z,"33 km S of Mammoth, Wyoming",earthquake,0.34,0.580,0.172572,4.0,reviewed,uu,uu
98,2025-11-11T01:50:19.032Z,-7.213000,129.454200,147.660,4.30,mb,21.0,103.0,2.710000,0.89,...,2025-11-11T03:31:39.040Z,"Kepulauan Babar, Indonesia",earthquake,9.50,9.136,0.133000,16.0,reviewed,us,us


In [18]:
data["country"] = data["place"].str.split(",").str[-1]

In [19]:
ranking = data["country"].value_counts().head(20)
print(ranking)


country
CA             2352
Alaska         1922
Texas           578
Hawaii          191
Washington      190
New Mexico      170
Oklahoma        166
Montana         153
Russia          147
Nevada          140
Wyoming         103
Puerto Rico     102
Philippines      81
Japan            81
Utah             79
Indonesia        79
Idaho            50
Chile            42
Oregon           39
Canada           36
Name: count, dtype: int64
